In [1]:
import sapscript as ss
import pyperclip
import os
import numpy as np
from datetime import datetime
from datetime import timedelta

In [34]:
OUT_DIR             = 'C:\OMNIMAIL\OUT\\'
IN_DIR              = 'C:\OMNIMAIL\IN\\'

E_PLANT = '7030' #ссылочный завод
E_STORAGE = '0023' #склад для агента RED
PARAMS = {'ERD':{'plant':'2059',
                 'agent':'100001589',  # 100001589 RED или 100004346 CSE
                 'agent_id': 'RED',
                 'idoc_port': 'SAPERD',
                 'rcpartner': 'ERDCLNT300'}, 
          'ERT':{'plant':'3024',
                 'agent':'100001514', # 100001514 RED или 100004238 CSE
                 'agent_id': 'Red',
                 'idoc_port': 'SAPERT',
                 'rcpartner': 'ERTCLNT100'}} 
ORDERS_CREATED_FILE = 'orders_created.txt' # берем отсюда номера ZIMG заказов, по которым планируем делать возвраты
RETURN_EXT_NUM_FILE = 'return_ext_num.txt' # счетчик внешнего номера возврата от Агента


class Returns:
    def __init__(self, system_id):
        self.sys_id = system_id
        self.sap = ss.sap()
        self.sap.check_system(self.sys_id)
        self.ext_number = [] # Внешние номера возврата Агента
        self.orders_created = [] # ZIMG заказы, участвующие в возвратах
        self.return_pos = [] # Позиции возвратов. Строка: [Заказ, Товар, Количество возврата]
        self.today = datetime.now().strftime('%Y%m%d')
        self.tomorrow = (datetime.now() + timedelta(days=1)).strftime('%Y%m%d')
        
    def zim_reestr(self):
        try:
            self.sap.run('zim_reestr')
            session = self.sap.session()
            session.findById("wnd[0]/usr/ctxtP_WERKS").text = E_PLANT
            session.findById("wnd[0]/usr/ctxtS_LGORT-LOW").text = E_STORAGE
            session.findById("wnd[0]/usr/ctxtS_WRKOUT-LOW").text = PARAMS[self.sys_id]['plant']
            
            # считываем из файла созданные ZIMG заказы, по которым планируем делать возвраты
            with open(ORDERS_CREATED_FILE, 'r') as file:
                self.orders_created = file.read().split('\n')
                # Убираем пустые строки из списка заказов
                self.orders_created = list(filter(lambda x: x,  self.orders_created))
            if len(self.orders_created)==0:
                raise ValueError('В файле не указаны ZIMG заказы.')
            
            print('ZIMG заказы для возвратов:')
            for order_num in self.orders_created:
                print(order_num)
                
            # Вставляем из буфера заказы на сел.экране
            pyperclip.copy('\r\n'.join(self.orders_created))
            session.findById("wnd[0]/usr/btn%_S_KDAUF_%_APP_%-VALU_PUSH").press()
            session.findById("wnd[1]/tbar[0]/btn[16]").press()
            session.findById("wnd[1]/tbar[0]/btn[24]").press()
            session.findById("wnd[1]/tbar[0]/btn[8]").press()
            
            session.findById("wnd[0]").sendVKey(8)
            
            # Считываем из ALV E-запасы по найденным выше заказам
            grid = session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")
            stocks = self.sap.read_alv(grid, cols = ["MAT_KDAUF", "MATNR", "MENGE"], tech_names=True)
            stocks.drop(index=stocks[stocks['MAT_KDAUF']==''].index, inplace=True)
            self.e_stock = stocks.sort_values(by='MAT_KDAUF', ascending=False)
        except:
            self.sap.error_print()
        finally:
            session = None
    
    def form_idoc(self, _e_stock_for_order, quantity, send):
        # Читаем текущий и записываем новый внешний номер возврата Агента из файла
        with open(RETURN_EXT_NUM_FILE, 'r') as file:
            ext_num = file.read().split('_')
            self.ext_number.append('_'.join(ext_num))
            ext_num[2] = str(int(ext_num[2])+1)
            ext_num_new = '_'.join(ext_num)
        with open(RETURN_EXT_NUM_FILE, 'w') as file:
            file.write(ext_num_new)
        
        try:
            self.sap.run('we19')
            session = self.sap.session()
            session.findById("wnd[0]/usr/radMSED7START-SEL_IDOCTP").select()
            session.findById("wnd[0]/usr/ctxtMSED7START-IDOCTYP").text = "ZIMRETAG"
            session.findById("wnd[0]/usr/ctxtMSED7START-IDOCTYP").setFocus()
            session.findById("wnd[0]/usr/ctxtMSED7START-IDOCTYP").caretPosition = 8
            session.findById("wnd[0]/tbar[1]/btn[8]").press()
            session.findById("wnd[0]/usr/lbl[9,1]").setFocus()
            session.findById("wnd[0]/usr/lbl[9,1]").caretPosition = 7
            session.findById("wnd[0]").sendVKey(2)
            session.findById("wnd[1]/usr/ctxtG_CONTROL_RECORD-RCVPOR").text = PARAMS[self.sys_id]['idoc_port']
            session.findById("wnd[1]/usr/ctxtG_CONTROL_RECORD-RCVPRN").text = PARAMS[self.sys_id]['rcpartner']
            session.findById("wnd[1]/usr/ctxtEDI_PRT-RCVPRT").text = "LS"
            session.findById("wnd[1]/usr/ctxtG_CONTROL_RECORD-SNDPOR").text = 'SAPPI'
            session.findById("wnd[1]/usr/ctxtG_CONTROL_RECORD-SNDPRN").text = PARAMS[self.sys_id]['agent']
            session.findById("wnd[1]/usr/ctxtEDI_PRT-SNDPRT").text = "LI"
            session.findById("wnd[1]/usr/ctxtEDI_MSGTYP-MSGTYP").text = "ZIMRETAG"
            session.findById("wnd[1]/usr/ctxtEDI_MSGTYP-MSGTYP").setFocus()
            session.findById("wnd[1]/usr/ctxtEDI_MSGTYP-MSGTYP").caretPosition = 8
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            session.findById("wnd[0]/usr/lbl[36,3]").setFocus()
            session.findById("wnd[0]/usr/lbl[36,3]").caretPosition = 6
            
            session.findById("wnd[0]").sendVKey(2)
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[0,21]").text = PARAMS[self.sys_id]['agent_id']
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[1,21]").text = self.today
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[2,21]").text = self.ext_number[-1]
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[3,21]").text = self.tomorrow
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[4,21]").text = PARAMS[self.sys_id]['plant']
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[4,21]").setFocus()
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[4,21]").caretPosition = 4
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            
            # Копируем сегменты под нужное кол-во позиций
            for pos in range((len(_e_stock_for_order)-1)):
                session.findById("wnd[0]/tbar[1]/btn[5]").press()
                session.findById("wnd[1]/usr/txt[5,15]").text = "ZIMRETAG_POSITION"
                session.findById("wnd[1]/usr/txt[5,15]").setFocus()
                session.findById("wnd[1]/usr/txt[5,15]").caretPosition = 0
                session.findById("wnd[1]/tbar[0]/btn[0]").press()  
                
            # Заполняем данные сегментов по позициям
            screen_index = 4 # индекс первой позиции на экране сегментов
            for index in range(len(_e_stock_for_order)):
                # вычисляем количество возврата для позиции в зависимости от режима работы
                if quantity == 'Full':
                    menge = _e_stock_for_order.iloc[index]['MENGE']
                elif quantity == 'One':
                    menge = 1
                elif quantity == 'Random':
                    menge = np.random.randint(0, _e_stock_for_order.iloc[index]['MENGE']+1)
                    if menge == 0:
                        # Удаляем сегмент с нулевым количеством
                        session.findById("wnd[0]/usr/lbl[8,"+str(screen_index)+"]").setFocus()
                        session.findById("wnd[0]/usr/lbl[8,"+str(screen_index)+"]").caretPosition = 0
                        session.findById("wnd[0]/tbar[1]/btn[14]").press()
                        continue
                
                matnr = _e_stock_for_order.iloc[index]['MATNR']
                session.findById("wnd[0]/usr/lbl[36,"+str(screen_index)+"]").setFocus()
                session.findById("wnd[0]/usr/lbl[36,"+str(screen_index)+"]").caretPosition = 0
                session.findById("wnd[0]").sendVKey(2)   
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[0,21]").text = _e_stock_for_order.iloc[index]['MAT_KDAUF']
                
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[1,21]").text = matnr
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[2,21]").text = menge
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[2,21]").setFocus()
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[2,21]").caretPosition = 1
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
                screen_index +=1
                
                # Записываем итоговую позицию возврата с получившимся кол-вом
                return_pos = [_e_stock_for_order.iloc[index]['MAT_KDAUF'], matnr, int(menge)]
                self.return_pos.append(return_pos)
            
            if send == True:
                # Отправляем айдок в порт
                session.findById("wnd[0]/tbar[1]/btn[8]").press()
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
                message = session.findById("wnd[2]/usr/txtMESSTXT1").text
                session.findById("wnd[2]/tbar[0]/btn[0]").press()
                
                print('WE19: Внешний номер возврата Агента - {0}'.format(self.ext_number[-1]))
                print('WE19:', message)
        except:
            self.sap.error_print()
        finally:
            session = None
            
    def we19(self, e_stock=None, quantity='Full', send=True, one_idoc=True):
        """Создание айдока ZIMRETAG на каждый отдельный заказ
        Параметры:
        e_stock  - датафрейм из метода zim_reestr, по умолчанию берем из self.e_stock
        quantity - параметр, отвечающий за кол-во в возврате:
                   'Full'   - полный возврат всех позиций;
                   'Random' - случайное число, не превышающее кол-во из заказа;
                   'One'    - единица
        send     - флаг: отправлять айдок в порт (False - оставить неотправленным и открытым на редактирование)
        one_idoc - флаг: True - формируем один айдок на все заказы
                         False - формируем на каждый заказ по одному айдоку"""
        
        # Если e_stock явно не задан, то выполняем расчеты по внутреннему параметру self.e_stock
        if len(ret.e_stock) > 0 and (not e_stock):
            e_stock = self.e_stock
            
        if len(e_stock)==0:
            raise ValueError('Не найдены заказы для возвратов.')
            
        if one_idoc == True:
            self.form_idoc(e_stock, quantity, send)
        else:   
            # По каждому заказу создаем айдок 
            _orders_list = list(set(e_stock['MAT_KDAUF']))
            for order_num in _orders_list:
                # Выделяем в общем списке е-запаса запас под конкретный заказ
                e_stock_for_order = e_stock[e_stock['MAT_KDAUF']==order_num]
                self.form_idoc(e_stock_for_order, quantity, send)
            
    def zimr(self, ext_num=None):
        try:
            self.sap.run('zshop_gr')
            session = self.sap.session()
            try:
                session.findById("wnd[0]/usr/ctxtGS_T001W-WERKS").text = PARAMS[self.sys_id]['plant']
            except:
                None
            session.findById("wnd[0]/usr/radGS_SCREEN-2430_NEXTDAY").select()
            session.findById("wnd[0]/tbar[1]/btn[8]").press()
            try:
                session.findById("wnd[0]/usr/ctxtS_WRK-LOW").text = PARAMS[self.sys_id]['plant']
            except:
                None
            # Указываем внешние номера заявок на возврат, если они есть
            if ext_num != None:
                pyperclip.copy('\r\n'.join(ext_num))
                session.findById("wnd[0]/usr/btn%_S_BOLNR_%_APP_%-VALU_PUSH").press()
                session.findById("wnd[1]/tbar[0]/btn[24]").press()
                session.findById("wnd[1]/tbar[0]/btn[8]").press()
            session.findById("wnd[0]/tbar[1]/btn[8]").press()
        except:
            self.sap.error_print()
        finally:
            session = None
    
    def form_tsd_file(self, defect=False, random=False):
        try:
            session = self.sap.session()
            # Читаем данные по выгруженным позициям из нижней ALV реестра возвратов
            _grid = session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[1]/shell")
            self.cur_return_pos = self.sap.read_alv(_grid, cols = ["TKNUM", "VBELN", "MATNR", "ORMNG"], tech_names=True)
        except:
            self.sap.error_print()
        finally:
            session = None
            
        _transport = self.cur_return_pos['TKNUM'].unique()[0]
        _shtr_file = OUT_DIR+"[Возврат_от_агентов-SHTR]"+_transport+".DAT"
        
        if not os.path.exists(_shtr_file):
            print('Данные не выгружены в ТСД')
            return
        
        # Читаем файл со штрих-кодами товаров
        with open(_shtr_file) as file:
            _shtr_matnr = file.read().split('\n')
        _shtr_matnr = list(filter(lambda x: x, _shtr_matnr)) #Удаляем пустые строки
        _shtr_matnr = [string.split(',') for string in _shtr_matnr]
        
        # Записываем последний ШК для каждого из товаров в словарь
        self.shtr_matnr_dict = {key: value for (key, value) in _shtr_matnr}
        self.cur_return_pos['EAN'] = self.cur_return_pos['MATNR'].apply(lambda matnr: self.shtr_matnr_dict[matnr])
        
        # Записываем файл для загрузки в ТСД
        _in_file = IN_DIR+"[Возврат_от_агентов-RESULT]"+_transport+".DAT"
        with open(_in_file, 'w+') as file:
            for index, row in self.cur_return_pos.iterrows():
                _quantity = int(row['ORMNG'])
                
                # Если принимаем с браком
                _defect_text = ''
                if defect == True:
                    _defect_qnty = np.random.randint(0,_quantity+1)
                    if _defect_qnty > 0:
                        _quantity = _quantity - _defect_qnty
                        _defect_type = str(np.random.randint(1,4))
                        _defect_text = '"'+row['VBELN']+'",'+row['EAN']+','+row['MATNR']+','+str(_defect_qnty)+',B,"'+_defect_type+'"\n'
                
                # Если хотим случайные излишки и недостачи
                if random == True:
                    _quantity = max(0,_quantity+int(np.random.normal(scale=2)))
                text = '"'+row['VBELN']+'",'+row['EAN']+','+row['MATNR']+','+str(_quantity)+',G,""\n'+_defect_text
                
                file.write(text)
        print('Записан файл для ТСД:', _in_file)

## Возвраты в ERD/ERT

In [36]:
ret = Returns('ERD')

# Считываем запасы агента по ранее созданным и отгруженным заказам
ret.zim_reestr()

# Создаем IDOC вида ZIMRETAG на каждый заказ из e_stock
ret.we19(quantity='Full', send=True, one_idoc=True)

# Открываем ZIMR со списком созданных заявок на возврат
#ret.zimr(ext_num=ret.ext_number)

ZIMG заказы для возвратов:
45961913
45961914
WE19: Внешний номер возврата Агента - EXT_NUM_10205
WE19: Номер IDOC 0000000001392442 передан в приложение.


In [33]:
len(ret.e_stock)

8

In [5]:
# Формирование файла для загрузки из ТСД
#ret = Returns('ERD')
ret.form_tsd_file(defect=True, random=True)

Записан файл для ТСД: C:\OMNIMAIL\IN\[Возврат_от_агентов-RESULT]2000001915.DAT


In [585]:
e_stock

,MAT_KDAUF,MATNR,MENGE
8,5630056989,1000054081,2.0
9,5630056989,1000054083,2.0
10,5630056989,1000054388,2.0
11,5630056989,1000007820,2.0
4,5630056988,1000054081,2.0
5,5630056988,1000054083,2.0
6,5630056988,1000054388,2.0
7,5630056988,1000007820,2.0
0,5630056987,1000054081,2.0
1,5630056987,1000054083,2.0
